Projekt ma na celu rozwiązanie problemu rozpoznawania i klasyfikacji poszczególnych słow w nieustrukturyzowanym tekscie. 

Przedstawione zostały różne metody, począwszy od podstawowych technik klasyfikacji, które nie dają wystarczająco zadowalających rezultatów, a skończywszy na na bardziej zaawansowanej metodzie, która daje dobre wyniki ewaluacji.

In [1]:
import eli5
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report

from FeatureExtractor import FeatureExtractor
from SentenceExtractor import SentenceExtractor

In [2]:
import warnings
warnings.filterwarnings('ignore')

Dane pochodzą z serwisu Kaggle, który zawiera bardzo wiele danych nadających się do wykorzystania w uczeniu maszynowym.

In [19]:
data_frame = pd.read_csv('../ner_dataset.csv', encoding='ISO-8859-1')
data_frame = data_frame[:65000]

Plik z danymi zawiera ciąg wyrazów, z których każdy opatrzony jest pewną charakterystyką.
Wyrazy są pogrupowane w w poszczególne zdania. Każdy wyraz zawiera etykiety "POS" oraz "Tag".

"POS" (part of speech) to etykieta, który informuje o tym jaką cześcią mowy jest dane słowo. Dla słow wylistowanych poniżej, mamy np. etykieta NNS - oznacza on iż jest to rzeczownik w liczbie mnogiej. Z kolei np. VBN oznacza czasownik w czasie przeszłym (imiesłów bierny). 

Wyjaśnienie każdego z tych etykiet można znaleźć na przykład pod adresem: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

"Tag" to etykieta mówiąca o tym czym jest dane słowo oraz w jakim miejscu w ciągu wyrazów się znajduje. W ramach tego zestawu danych, mamy do czynienia z następującymi rodzajami słow:

geo = Geographical Entity,
org = Organization,
per = Person,
gpe = Geopolitical Entity,
tim = Time indicator,
art = Artifact,
eve = Event,
nat = Natural Phenomenon

Za wskazanie kontekstu miejsca w ciągu wyrazów odpowiada format IOB (inside, outside, beginning).
"I" oznacza, że dane słowo jest w środku łańcucha,
"B" - dane słowo jest początkiem łańcucha wyrazów,
"O" - dane słowo nie należy do żadnego łańcucha.

"Tag" będzie naszą "szukaną" - inaczej mówiąc, dla poszczegolnych słow, będziemy starali się odnaleźć odpowiedni tag. Dzięki temu będziemy wiedzieli czym jest dane słowo, np. osobą czy może organizacją albo adresem.

In [5]:
data_frame.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [6]:
'Number of NaN values: ', data_frame.isnull().sum()

('Number of NaN values: ', Sentence #    85902
 Word              0
 POS               0
 Tag               0
 dtype: int64)

Rekordy z wartościami NaN możemy zamienić na poprzednie wartości.
W obecnej ramce danych mamy 2942 unikalnych zdań, 8686 słow i 17 rodzajów tagów.

In [20]:
data_frame = data_frame.fillna(method='ffill')

data_frame['Sentence #'].nunique(), data_frame.Word.nunique(), data_frame.Tag.nunique()

(2942, 8686, 17)

Aby wykonać klasyfikację klasycznymi metodami, potrzebujemy przedstawić tekst w formie wektora. W tym celu używamy biblioteki sklearn - DictVectorizer.
Zmienną objaśnianą (y) będą tagi a zmienne objaśniające (X) to pozostałe kolumny w ramce danych.

In [21]:
# ==== TRANSFORMING TEXT TO VECTOR ====
vector = DictVectorizer(sparse=False)

In [22]:
# splitting data frame into data and class columns
X = data_frame.drop('Tag', axis='columns')
X = vector.fit_transform(X.to_dict('records'))
y = data_frame.Tag.values

In [23]:
# ==== TRAIN-TEST SPLIT ====
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Pierwszą metodą klasyfikacji będzie Random Forest.

Na początku przedstawimy każde ze słow jako zbiór charakterystyk zebranych do postaci tablicy. Owymi charakterystykami będą: długość słowa, flaga oznaczająca czy słowo jest liczbą, flaga wskazująca czy słowo jest pisane z wielkich liter i kolejna flaga dla małych liter.

In [24]:
# ==== CLASSIFICATION USING REGULAR CLASSIFIERS ====

# ** Random Forest **

# very basic, naive approach - we assume the word has a features like if it's an uppercase, title and so on
def get_feature_from_word(word: str):
    return np.array([len(word), word.isdigit(), word.isupper(), word.islower()])


words = [get_feature_from_word(word) for word in data_frame["Word"].values.tolist()]
# tags are classes in the model
tags = data_frame["Tag"].values.tolist()
rf = RandomForestClassifier(n_estimators=20)
rf_y_pred = cross_val_predict(rf,  X=words, y=tags)

print(classification_report(y_pred=rf_y_pred, y_true=tags))

             precision    recall  f1-score   support

      B-art       0.00      0.00      0.00        53
      B-eve       0.00      0.00      0.00        45
      B-geo       0.21      0.66      0.32      2040
      B-gpe       0.15      0.12      0.13      1207
      B-nat       0.00      0.00      0.00        20
      B-org       0.47      0.17      0.25      1219
      B-per       0.00      0.00      0.00      1090
      B-tim       0.29      0.25      0.27      1148
      I-art       0.00      0.00      0.00        34
      I-eve       0.00      0.00      0.00        37
      I-geo       0.00      0.00      0.00       409
      I-gpe       0.00      0.00      0.00        34
      I-nat       0.00      0.00      0.00         9
      I-org       0.25      0.02      0.04       909
      I-per       0.10      0.00      0.00      1216
      I-tim       0.53      0.08      0.13       329
          O       0.96      0.98      0.97     55201

avg / total       0.85      0.86      0.85  

Jak widać wyniki są bardzo marne. Było to spodziewane, gdyż charakterystki jakie były brane pod uwage, są zdecydowanie zbyt ogólnikowe i nie zawierają wystarczającej liczby informacji.

Dodatkowo, dla kolejnej klasyfikacji postanowiono usunąć tag "O". Jest on tagiem, który pojawia się zdecydowanie zbyt często, czym zaburza średnią wyników przy ewaluacji.

In [25]:
# ** Multi-layer Percepton **
perceptron = Perceptron(verbose=20, n_jobs=-1)
tags = np.unique(y)
tags = tags.tolist()
# use of partial_fit (out-of-core algorithm) to process data with limited amount of RAM
perceptron.partial_fit(X_train, y_train, tags)

# remove the most common tag 'O' not to disturb evaluation metrics
new_tags = tags.copy()
del new_tags[-1]

perceptron_y_pred = perceptron.predict(X_test)
# classification report
print(classification_report(y_pred=perceptron_y_pred, y_true=y_test, labels=new_tags))

-- Epoch 1
Norm: 12.17, NNZs: 137, Bias: -4.000000, T: 45500, Avg. loss: 0.001802
Total training time: 1.09 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


Norm: 11.18, NNZs: 101, Bias: -3.000000, T: 45500, Avg. loss: 0.001121
Total training time: 0.94 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    2.0s remaining:    0.0s


Norm: 56.66, NNZs: 1866, Bias: -4.000000, T: 45500, Avg. loss: 0.041033
Total training time: 0.96 seconds.
-- Epoch 1


...

Norm: 27.13, NNZs: 504, Bias: -4.000000, T: 45500, Avg. loss: 0.011516
Total training time: 0.92 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   15.1s remaining:    0.0s


Norm: 60.47, NNZs: 1977, Bias: 3.000000, T: 45500, Avg. loss: 0.047582
Total training time: 0.93 seconds.


[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:   16.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:   16.1s finished


             precision    recall  f1-score   support

      B-art       0.50      0.08      0.13        13
      B-eve       0.50      0.31      0.38        16
      B-geo       0.60      0.74      0.66       589
      B-gpe       0.63      0.78      0.69       344
      B-nat       0.14      0.40      0.21         5
      B-org       0.51      0.46      0.49       386
      B-per       0.60      0.54      0.57       324
      B-tim       0.96      0.64      0.77       341
      I-art       1.00      0.10      0.18        10
      I-eve       0.64      0.54      0.58        13
      I-geo       0.66      0.31      0.42       145
      I-gpe       0.00      0.00      0.00        10
      I-nat       0.50      0.33      0.40         3
      I-org       0.32      0.61      0.42       286
      I-per       0.75      0.23      0.35       357
      I-tim       0.35      0.30      0.32        90

avg / total       0.62      0.55      0.55      2932



Metoda Multi-Layer Percepton daje o wiele lepsze rezultaty. Średnia wyników jest oczywiście niższa ale na tym etapie jest już pozbawiona zaburzenia tagiem "O".

Zerowy F1 dostajemy już tylko dla tagu "gpe", w poprzedniej metodzie, z zerami mieliśmy do czynienia przy większości tagów.

In [26]:
# ** Stochastic Gradient Descend **

sgd = SGDClassifier()
sgd.partial_fit(X_train, y_train, tags)
sgd_y_pred = sgd.predict(X_test)
print(classification_report(y_pred=sgd_y_pred, y_true=y_test, labels=new_tags))

             precision    recall  f1-score   support

      B-art       0.00      0.00      0.00        13
      B-eve       0.25      0.06      0.10        16
      B-geo       0.69      0.65      0.67       589
      B-gpe       0.72      0.62      0.67       344
      B-nat       0.33      0.20      0.25         5
      B-org       0.29      0.62      0.40       386
      B-per       0.68      0.50      0.57       324
      B-tim       0.68      0.65      0.66       341
      I-art       0.00      0.00      0.00        10
      I-eve       0.42      0.62      0.50        13
      I-geo       0.56      0.45      0.50       145
      I-gpe       0.00      0.00      0.00        10
      I-nat       0.00      0.00      0.00         3
      I-org       0.76      0.31      0.44       286
      I-per       0.79      0.24      0.37       357
      I-tim       0.22      0.10      0.14        90

avg / total       0.63      0.50      0.53      2932



Z kolei metoda stochastycznego gradientu stochastycznego daje podobne rezultaty, jednak w tym przypadku odrobinę gorsze.

Jedną z najlepszych metod używanych w procesach Name Entity Recognition jest metoda Conditional Random Fields. Jest ona dużo bardziej skomplikowana, wymaga również większego preprocesingu danych. Jednak rezultaty najczęściej są dużo bardziej interesujące.

Aby móc wykorzystać metodę CRF, utworzono dwie klasy pomocnicze: SentenceExtractor oraz FeatureExtractor.

SentenceExtractor pozwala na wyciągnięcie z ramki danych zdań składających się z słow wraz z ich etykietami czyli POSem i Tagiem.
FeatureExtractor pozwala na stworzenie struktury charakterystych dla każdego zdania. Charakterystyki te nie są jednak tak podstawowe, jakie zostały stworzone przy okazji metody Random Forest. Zawierają one np. konkretne fragmenty słów czy tagi POS oraz ich same fragmenty. Całośc stanowi dość szeroką strukturę featureów każdego słowa.

In [27]:
# ==== CLASSIFICATION USING CONDITIONAL RANDOM FIELDS ====

sentence_extractor = SentenceExtractor("Sentence #")
labeled_sentences = sentence_extractor.extract(data_frame)

feature_extractor = FeatureExtractor()
X = feature_extractor.sentences2features(labeled_sentences)
y = feature_extractor.sentences2labels(labeled_sentences)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

crf = CRF(algorithm='lbfgs', c1=10, c2=0.1, max_iterations=100, all_possible_transitions=False)
crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
print(flat_classification_report(y_test, y_pred, labels=new_tags))

             precision    recall  f1-score   support

      B-art       0.00      0.00      0.00        21
      B-eve       0.00      0.00      0.00        10
      B-geo       0.59      0.88      0.70       623
      B-gpe       0.82      0.73      0.77       375
      B-nat       0.00      0.00      0.00         3
      B-org       0.73      0.46      0.57       426
      B-per       0.75      0.72      0.74       360
      B-tim       0.94      0.63      0.75       402
      I-art       0.00      0.00      0.00        11
      I-eve       0.00      0.00      0.00         9
      I-geo       0.58      0.44      0.50       130
      I-gpe       0.00      0.00      0.00        10
      I-nat       0.00      0.00      0.00         1
      I-org       0.64      0.62      0.63       292
      I-per       0.77      0.91      0.84       401
      I-tim       0.69      0.36      0.47       111

avg / total       0.72      0.68      0.68      3185



Jak widać, rezultaty pozyskane z metody CRF różnią się znacząco od klasycznych metod klasyfikacji. Obserwujemy poprawę wyników niemal w każdej klasie i jest to poprawa znacząca.

Wynik zwracane przez klasę CRF z pakietu sklearn dają spore pole do eksploracji tego, czego nauczył się algorytm.

Na przykład, właściwość state_features_ pozwala na interesującą interpretację, jakiej klasy będzie słowo znajdujące się obok danego. Poniższy kod prezentuje największe i najmniejsze prawdopodobieństwa.

In [28]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.4f %-8s %s" % (weight, label, attr))


print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(10))
print("\n Top negative:")
print_state_features(Counter(crf.state_features_).most_common()[-10:])


Top positive:
4.8833 O        bias
3.4775 O        word.lower():minister
3.4106 B-tim    word[-3:]:day
2.9777 B-gpe    word.istitle()
2.8020 B-tim    word[-2:]:ay
2.3043 B-tim    -1:word.lower():in
2.1842 O        BOS
2.0706 B-gpe    word[-3:]:ans
2.0096 B-geo    -1:word.lower():in
1.6985 O        postag[:2]:VB

 Top negative:
-0.8840 B-per    -1:postag[:2]:DT
-0.9720 I-per    +1:postag[:2]:NN
-1.0897 I-tim    word.istitle()
-1.2392 O        +1:word.lower():years
-1.4374 B-geo    -1:postag[:2]:NN
-1.5311 O        postag:JJ
-1.7605 O        word.isdigit()
-2.3203 O        postag:NNPS
-2.4537 O        word.istitle()
-4.4068 O        postag:NNP


Z powyższych wyników można zauważyć, że np. słowo, które kończy się na "day" jest słowem oznaczającym czas. Z kolei jeżeli mamy do czyniania ze słowem "in", to kolejnym słowem będzie nazwa geograficzna.

Klasa CFR ma jeszcze wiele innych właściwości, które można eksplorować do innych celów. Jedną z ciekawszych jest właściwość transition_features_, która określa prawdopodobieństwo klasy w jakiej znajdzie się następne słowo, biorąc pod uwagę jakiej klasy jest słowo poprzednie.

Powyższe właściwości można też przedstawić graficznie za pomocą pakietu ELI5.
Tabela poniżej, prezentuje właśnie tranzycje między dwoma klasami słow.

In [33]:
eli5.show_weights(crf, top=30)

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,2.607,0.807,0.0,0.837,0.0,1.76,0.0,1.083,0.0,0.0,0.0,1.7,0.0,1.423,0.0,1.691,0.0
B-art,0.0,0.0,3.493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-art,-0.018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-eve,-0.0,0.0,0.0,0.0,4.357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-eve,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-geo,0.608,0.0,0.0,0.0,0.0,0.0,5.593,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.837,0.0
I-geo,0.0,0.0,0.0,0.0,0.0,0.0,3.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-gpe,0.056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.677,0.0,0.0,0.0,0.0,0.196,0.0,0.0,0.0
I-gpe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-nat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
